In [7]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy import special
import random

So, we have seen that the MCMC method is very good but it takes a while because we are starting from quite a few poisitions. The BRUTE method by comparison is quicker but less accurate. I think that using the BRUTE method to quickly find a few good starting poisitions for the MCMC would be the most efficient method.

We want this to be efficient since the plan is to input many different matrix elements to work out the best transitions to use.

In [8]:
#Define all of the frequencies we care about (we are working in MHz and are ignoring the 2pi for now to make things easy - also, frequencies are rounded) - these numbers don't actually matter.
#The setup is a bit difficult, so everything is carefully laid out:

DwG = 8000      #Ground state splitting
DwM = 60        #Metastable state splitting
Dw = 170.26e6   #Difference between the lowest metastable state and the highest ground state

#We are defining the frequency to be sitting above the 
Dw1 = 7000      #Where the unmodulated signal is sitting above the lowest ground state

#Define the frequency at which we have parked the beam - this is the most general way of defining the beam since
w = (DwG - Dw1) + Dw 

#Define the frequencies that we wish to find:
w1 = Dw
w2 = Dw + DwM
w3 = Dw + DwG
w4 = Dw + DwG + DwM

#Define the frequencies we wish to modulate at:
W = [(DwG - Dw1),Dw1,DwM]

Order = 2   #Considering up to the 2nd order.

M = [1,1.1,0.9,1]   #the matrix elements for each transition

In [25]:
def COMBO(Order,W,M,Rinitial,Spacings,NumberOfPoints,Betas,Props,N,StepSize):

    Index_history = []
    Func_history = []
    Diff_history = []
    Eff_history = []

    #define variables that are used within this function:

    ###First step is to define all of the possible combinations from the given frequencies and then work out the amplitudes for each tone
    #Work out how to make w1,w2,w3 and w4 using wm1,wm2 and wm3 from w. Consider only O orders.
    values = range(-Order, Order + 1)  # from -Order to Order

    Mod = []
    #intertools produces all possible combinations of elements
    for p in itertools.product(values, repeat = np.size(W)):     # this creates a list of all the possible combinations of each modulation frequency: we are making p a list rather than a tuple
        Mod.append(p)

    #filter out all of the combinations that we won't use and create a list of all of the useful ones:
    Combinations = []       #Define a list of all of the combinations for each frequency. Each component will be: [frequency that this adds up to, no. of W[0],...,no. of W[N]]
    for list in Mod:

        #find the frequency when summing up the modulation frequencies as described by this component of Mod:
        freq = w
        for x in range(np.size(W)):
            freq += list[x] * W[x]

        #now, we want to see if this frequency is useful at all:
        if freq == w1:
            Combinations.append([1,list])
        if freq == w2:
            Combinations.append([2,list])
        if freq == w3:
            Combinations.append([3,list])
        if freq == w4:
            Combinations.append([4,list])

    #Define a function that outputs the amplitude for this sideband
    def A(list,ModIndices):
        
        A = 1
        for x in range(np.size(ModIndices)):
            A *= special.jv(list[x], ModIndices[x])
        return(A)

    #Define a function to find the amplitudes for each of the frequencies
    def FindAmps(Combinations,ModIndices):
        Amps = [0,0,0,0]

        for i in range(4):
            for x in Combinations:
                if x[0] == i+1:
                    Amps[i] += A(x[1],ModIndices)

        #Make the amplitudes absolute since their signs no longer matter (no more intereference):
        return(np.abs(Amps))
    
    #define a function to use these two previous functions to work out the maximum difference between the Rabi frequencies and the efficiency at these modulation indices
    def DiffAndEff(Combinations,ModIndices,M):
        Amps = FindAmps(Combinations,ModIndices)

        mean = 0
        for x in range(np.size(M)):
            mean += Amps[x] * M[x]      #this adds up all of the Rabi frequencies
        mean /= np.size(M)              #then divide at the end to get the mean

        #now, find the largest difference:
        Diff = []

        for x in range(np.size(M)):
            Diff.append(np.abs(( Amps[0] * M[0] - Amps[x] * M[x] ) / mean ))  #to speed things up, we are just considering the difference between one of the Rabi frequencies and the others

        #find the maximum rabi freq out of all of these:
        MaxDiff = np.max(Diff)

        #also work out the efficiency:
        efficiency = 0
        for x in range(np.size(Amps)):
            efficiency += Amps[x] ** 2

        return(MaxDiff,efficiency)

    ###Up to here has been the same for both functions. The next step is to define the MCMC 'energy' and to minimise it with the MCMC algorithm
    #this is the function:
    def func(Diff, Eff, prop):
        func = Diff * (1 - Eff) ** prop
        return(func)

    #and a function for our random steps:
    def choice(beta, func, Newfunc):
            prob = np.exp( beta * ( (func - Newfunc) / (0.5 * func + 0.5 * Newfunc))) # make this the mean so that as the function gets smaller, we still have random steps
            decision = random.choices([1 , 0] , [prob, 1-prob])
            return(decision[0])

    #first, we want to find a few good starting points using a brute force method:
    #define a function to find the initial points.
    def BestIndices(NumberOfPoints,R,spacing,Combinations):

        #define all of the points we are calculating values at for every point in this mesh
        X = np.linspace(R[0][0], R[0][1], int( (R[0][1] - R[0][0]) / (spacing) + 1) )
        Y = np.linspace(R[1][0], R[1][1], int( (R[1][1] - R[1][0]) / (spacing) + 1) )
        Z = np.linspace(R[2][0], R[2][1], int( (R[2][1] - R[2][0]) / (spacing) + 1) )

        AllIndices = [] #save all of the indices
        AllDiffs = []  #save all of the differences

        for x in range(np.size(X)):
            for y in range(np.size(Y)):
                for z in range(np.size(Z)):
                    Indices = [X[x],Y[y],Z[z]]       #the point we are considering
                    MaxDiff, _ = DiffAndEff(Combinations,Indices,M)
                    AllIndices.append(Indices)
                    AllDiffs.append(MaxDiff)

        # get indices of the three smallest diffs
        NextPoints = []
        for x in range(0,NumberOfPoints):
            index = AllDiffs.index(min(AllDiffs))
            NextPoints.append( AllIndices[index] )
            del AllDiffs[index]
            del AllIndices[index]

        return(NextPoints)       #returning the three points that had the smallest difference in Rabi frequencies

    #then immediately call this function:
    #find the initial points:
    ModIndices = BestIndices(NumberOfPoints,Rinitial,Spacings,Combinations)
    #define the MCMC for these variables:
    def MCMC_Variables(ModIndices,Beta,Prop,N,StepSize):

        #these are all the things we want to record:
        Diff_history = []
        Eff_history = []
        func_history = []
        Index_history = []

        #run the loops:
        for i in range(0,N): 
            #start off with adding to modulation indices, then subtract.
            for sign in range(0,2):

                #see if changing the modulation index by a small step does anything, change F accordingly
                for x in range(np.size(ModIndices)):
                    
                    #make a copy of the indices, modified by the step
                    NewModIndices = ModIndices.copy()
                    NewModIndices[x] += (-1) ** sign * StepSize

                    #find the difference and efficiency for each
                    Diff, Eff = DiffAndEff(Combinations,ModIndices,M)
                    F = func(Diff, Eff, Prop)

                    NewDiff, NewEff = DiffAndEff(Combinations,NewModIndices,M)
                    NewF = func(NewDiff, NewEff, Prop)

                    if NewF < F:
                        ModIndices = NewModIndices    #make the forwards step if we can
                        #save the new values (which may be the same as the old values)
                        func_history.append(NewF)
                        Diff_history.append(NewDiff)
                        Eff_history.append(NewEff)
                        Index_history.append(NewModIndices)
                    
                    else:
                        if choice(Beta,F,NewF) == 1:
                            ModIndices = NewModIndices    #make a backwards step sometimes, with more probability if the step is small
                            #save the new values (which may be the same as the old values)
                            func_history.append(NewF)
                            Diff_history.append(NewDiff)
                            Eff_history.append(NewEff)
                            Index_history.append(NewModIndices)
                        else:

                            #save the new values (which may be the same as the old values)
                            func_history.append(F)
                            Diff_history.append(Diff)
                            Eff_history.append(Eff)
                            Index_history.append(ModIndices)

        return(Index_history,func_history,Diff_history,Eff_history)

    #iterate over all points
    for p in ModIndices:
        #iterate over all betas
        for B in Betas:
            #iterate over all proportions
            for Prop in Props:
                    #run the algorithm for these variables
                    TempIndex_history,TempFunc_history,TempDiff_history,TempEff_history = MCMC_Variables(p,B,Prop,N,StepSize)
                    Index_history.append(TempIndex_history)
                    Func_history.append(TempFunc_history)
                    Diff_history.append(TempDiff_history)
                    Eff_history.append(TempEff_history)
    
    #flatten out these arrays:
    TempIndex_history, Func_history, Diff_history, Eff_history = np.array(Index_history), np.array(Func_history), np.array(Diff_history), np.array(Eff_history)
    TempIndex_history, Func_history, Diff_history, Eff_history =  TempIndex_history.flatten(), Func_history.flatten(), Diff_history.flatten(), Eff_history.flatten()

    #make the index history into the right shape
    n = int( np.size(ModIndices) )  # group size
    Index_history = [ TempIndex_history[i:i + n] for i in range(0, len(TempIndex_history), n)]

    return(Index_history, Func_history, Diff_history, Eff_history)

In [ ]:
#Call the MCMC function:

#Use code from the brute force method to convert a range and spacing into a lattice of indices
Rinitial = [ [0.1,1.8] , [0.1,1.8] , [0.1,1.8] ]
Spacings = 0.1
NumberOfPoints = 5

StepSize = 0.001
Betas = [3000,7000]
Props = [5,7]
N = 5*10**3

COMBOIndices, COMBOFuncs, COMBODiffs, COMBOEffs = COMBO(Order,W,M,Rinitial,Spacings,NumberOfPoints,Betas,Props,N,StepSize)

In [ ]:
###Select only the points that are best for us:
mask = COMBODiffs <= 0.001
COMBODiffsResults = COMBODiffs[mask]
COMBOEffResults =COMBOEffs[mask]

if min(COMBODiffs) < 0.001:
    print('COMBO has a result!')

print(min(COMBODiffs))

0.0033891825407461394
